<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/%5B210516%5Dsampling_with_PT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torch.utils.data
import torch.nn as nn
from itertools import combinations
from itertools import permutations
from datetime import datetime
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
# np.seterr(divide='ignore', invalid='ignore')
import itertools
import re
from time import sleep

In [3]:
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, k, use_cuda):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        if use_cuda==True:
            self.v = nn.Parameter(torch.ones(1, n_vis).cuda())
            self.h = nn.Parameter(torch.zeros(1, n_hid).cuda())
            self.W = nn.Parameter(torch.randn(n_hid, n_vis).cuda())
            self.k = k
        else:
            self.v = nn.Parameter(torch.ones(1, n_vis))
            self.h = nn.Parameter(torch.zeros(1, n_hid))
            self.W = nn.Parameter(torch.randn(n_hid, n_vis))
            self.k = k     

    def visible_to_hidden(self, v, beta):
        return torch.sigmoid(F.linear(v, self.W, self.h)*beta)

    def hidden_to_visible(self, h, beta):
        return torch.sigmoid(F.linear(h, self.W.t(), self.v)*beta)

    def free_energy(self, v):
        v_term = torch.matmul(v, self.v.t())
        w_x_h = F.linear(v, self.W, self.h)
        h_term = torch.sum(F.softplus(w_x_h), dim=1)
        return torch.mean(-h_term - v_term)
    
    def energy2(self, v, h):
        return -torch.matmul(v, self.v.t())-torch.matmul(torch.matmul(v, self.W.t()),h.t())-torch.matmul(h, self.h.t())

    def forward(self, v):
        h = self.visible_to_hidden(v)
        h = h.bernoulli()
        for _ in range(self.k):
            v_gibb = self.hidden_to_visible(h)
            v_gibb = v_gibb.bernoulli()
            h = self.visible_to_hidden(v_gibb)
            h = h.bernoulli()
        return v, v_gibb

In [4]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

if CUDA:
    device='cuda'
else:
    device='cpu'
torch.cuda.is_available()

False

In [5]:
CUDA

False

In [6]:
# 아래 두 함수는 PT와는 아무 상관 없습니다. 10진법을 2진법으로 바꾸고 리스트 형태로 변형하는 함수입니다. 예)15->[1,1,1,1]
# 그리고 그 다음은 역함수입니다.
def decimal_to_binary(integer):
    string=bin(integer)[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def binary_to_decimal(list0):
    value=0
    for i in range(len(list0)):
        value+=list0[-i-1]*2**(i)
    return value

# 이 함수는 PT에 사용되는 transition 확률을 계산합니다.
def swap_prob(i,j, model, list00, list11):
    v1=torch.tensor(list00[i]).view(1,n_vis)
    v2=torch.tensor(list00[j]).view(1,n_vis)
    h1=torch.tensor(list11[i]).view(1,n_hid)
    h2=torch.tensor(list11[j]).view(1,n_hid)
    beta1=beta_list[i]
    beta2=beta_list[j]
    return torch.exp((beta2-beta1)*(model.energy2(v2, h2)-model.energy2(v1, h1)))

# 이 함수는 tansition 확률에 의거, swap을 수행합니다.
def swap(list0, list1, model):
    k=np.random.randint(0, len(list0)-1)
    combination=[k, k+1]
    if swap_prob(combination[0], combination[1], model, list0, list1)>=np.random.rand():
        a=list0[combination[0]]
        b=list0[combination[1]] 
        list0[combination[0]]=b
        list0[combination[1]]=a
        
        c=list1[combination[0]]
        d=list1[combination[1]] 
        list1[combination[0]]=d
        list1[combination[1]]=c
    else: None
#     return list0, list1
#     combinations=list(itertools.combinations(list(range(len(list0))), 2))
#     for k in range(len(combinations)):
#         if swap_prob(combinations[k][1], combinations[k][0])>=np.random.rand():
#             list0[combinations[k][0]]=list0[combinations[k][1]]; list0[combinations[k][1]]=list0[combinations[k][0]]
#         else: None
#     return list0

def P_h(list0):
    config_count={}
    for i in range(len(list0)):
        config_count[str(int(binary_to_decimal(list0[i][0])))]=0
    for i in range(len(list0)):
        config_count[str(int(binary_to_decimal(list0[i][0])))]+=1
    return config_count

def prod(L):
    p=1
    for i in L:
        p= i * p
    return p

# def Estimate_Z(model0, states):
#     Z=0
#     for i in range(len(states[0])):
#         Z+=torch.exp(-model0.energy2(states[0][i], states[1][i])).detach()
#     return float(Z.detach().numpy())

def get_hist(list00, color='red'):
    bins=range(int(min(list00)-30), int(max(list00)+30), 1)
    y1,x1,_ = plt.hist(list00, bins = bins, histtype='step', color=color)
    x1 = 0.5*(x1[1:]+x1[:-1])
    return x1, y1


# $F=\ln[Q(\vec{h})\prod_i\sigma(-a_i-\sum_jW_{ij}h_j)]-\vec{b}\cdot\vec{h}$

In [11]:
def FE_mean_std(model0_dict, p):
    FE_list=[]
    for i in range(len(p)):
        # n_vis=len(model0_dict['v'][0]); n_hid=len(model0_dict['h'][0])
        integer=list(p.keys())[i]
        h=decimal_to_binary(int(integer)).to(device)
        a=model0_dict['v'].view(1,n_vis).detach()
        b=model0_dict['h'].view(1,n_hid).detach()
        W=model0_dict['W'].view(n_hid,n_vis).detach()
        
        FE=prod(torch.sigmoid(-F.linear(h, W.t(), a)).detach()[0])*100000
        FE=FE*p[integer]
        FE=FE/sum(list(p.values()))
        FE=torch.log(FE.detach())-torch.log(torch.tensor([[100000]]))-torch.dot(b[0],h[0].t())
        FE_list.append(FE.detach())
    return [torch.mean(torch.tensor(FE_list)).detach().numpy(), torch.std(torch.tensor(FE_list)).detach().numpy()]

def dE_mean_std(model0, model1, v_list, h_list):
    dE=[]
    for i in range(len(v_list)):
        E0=model0.energy2(v_list[i].float().detach(), h_list[i].float().detach())[0].detach()
        E1=model1.energy2(v_list[i].float().detach(), h_list[i].float().detach())[0].detach()
        dE.append(E1-E0)
    return [torch.mean(torch.tensor(dE)).numpy(), torch.std(torch.tensor(dE)).numpy()]

def FE_V(model0, v0):
    a=model0['v'].detach()
    b=model0['h'].detach()
    W=model0['W'].detach()
    values=[]
    for i in range(len(v0)):
        v_term = torch.matmul(v0[i].detach(), a.t()).detach()
        w_x_h = F.linear(v0[i].detach(), W, b).detach()
        h_term = torch.sum(F.softplus(w_x_h), dim=1).detach()
        values.append(torch.mean(-h_term - v_term).detach().numpy())
    return np.mean(values)

def sampling_with_PT(fullmodel, eq_step, n_step):
    # states for full model
    hidden_states_now0=[decimal_to_binary(2**n_hid-1)]*len(beta_list)
    visible_states_now0=[1.]*len(beta_list)

    coldest_hidden0=[]
    coldest_visible0=[]
    
    # equilibrium steps for full model
    for step in range(eq_step):
        for i in range(len(beta_list)):
            visible_states_now0[i]=fullmodel.hidden_to_visible(hidden_states_now0[i].to(device), beta_list[i]).bernoulli().detach()
            hidden_states_now0[i]=fullmodel.visible_to_hidden(visible_states_now0[i].to(device), beta_list[i]).bernoulli().detach()
    

    # Tasks with PT
    for step in tnrange(n_step):
        for i in range(len(beta_list)):
            # Gibbs sampling of fu
            visible_states_now0[i]=fullmodel.hidden_to_visible(hidden_states_now0[i].to(device), beta_list[i]).bernoulli().detach()
            hidden_states_now0[i]=fullmodel.visible_to_hidden(visible_states_now0[i].to(device), beta_list[i]).bernoulli().detach()
        swap(visible_states_now0, hidden_states_now0, fullmodel)
        coldest_hidden0.append(hidden_states_now0[0].int())
        coldest_visible0.append(visible_states_now0[0].int())

    return coldest_visible0, coldest_hidden0

In [ ]:
beta_list=torch.tensor(np.linspace(1,0,10).astype(int)).to(device)
n_vis=9
T='cri'
for n_hid in [2,4,6,8]:
    list_v=[]; list_h=[]
    for i in range(10):
        por='full'
        model1_dict=torch.load('drive/MyDrive/loss_IG/3*3/state_dict/2021-05-02_n_hid={n_hid}_{por}_T={T}{c1}'.format(n_hid=n_hid, por=por, T=T, c1=i), map_location=device)
        model1=RBM(n_vis=n_vis, n_hid=n_hid, k=1, use_cuda=CUDA)
        model1.load_state_dict(model1_dict)
        a0, b0=sampling_with_PT(model1, 100, 100000)
        list_v.append(a0)
        list_h.append(b0)
    with open('drive/MyDrive/loss_IG/3*3/samples/3*3_PT_0.1M_n_hid={n_hid}_T={T}.pkl'.format(n_hid=n_hid, T=T), 'wb') as f:
        pkl.dump([list_v, list_h], f)

In [ ]:
fullconfigs=torch.tensor(pd.read_pickle('drive/MyDrive/loss_IG/3*3/3*3_cri_full.pkl'))
n_vis=9
T='cri'
por='full'
PT={}
for n_hid in [1,2,4,6,8]:
    PT[str(n_hid)]=[]
    v_and_h=pd.read_pickle('drive/MyDrive/loss_IG/3*3/samples/3*3_PT_0.1M_n_hid={n_hid}_T={T}.pkl'.format(n_hid=n_hid, T=T))

    for j in tnrange(10):
        v_list=v_and_h[0]; h_list=v_and_h[1]
        model0_dict=torch.load('drive/MyDrive/loss_IG/3*3/state_dict/2021-05-02_n_hid={n_hid}_{por}_T={T}{c0}'.format(n_hid=n_hid, por=por, T=T, c0=j), map_location=device)
        model0=RBM(n_vis=n_vis, n_hid=n_hid, k=1, use_cuda=CUDA)
        model0.load_state_dict(model0_dict)
        PT[str(n_hid)].append({'FE{c0}'.format(c0=j):FE_mean_std(model0_dict, P_h(h_list[j]))})
        PT[str(n_hid)].append({'FE_V{c0}'.format(c0=j):FE_V(model0_dict, fullconfigs)})

        for k in range(10):
            if j!=k:
                por='full'
                model1_dict=torch.load('drive/MyDrive/loss_IG/3*3/state_dict/2021-05-02_n_hid={n_hid}_{por}_T={T}{c1}'.format(n_hid=n_hid, por=por, T=T, c1=k), map_location=device)
                model1=RBM(n_vis=n_vis, n_hid=n_hid, k=1, use_cuda=CUDA)
                model1.load_state_dict(model1_dict)
                PT[str(n_hid)].append({'dE{c0}{c1}'.format(c0=j,c1=k):dE_mean_std(model0, model1, v_list[j], h_list[j])})
    del v_and_h
with open('drive/MyDrive/loss_IG/3*3/data_PT_0.1M_{T}.pkl'.format(T=T), 'wb') as f:
    pkl.dump(PT, f)

In [ ]:
def get_listmk(epoch1):
    config_count={} # 각 hidden layer state 갯수 파악 (k)
    for i in range(len(epoch1)):
        config_count[str(epoch1[i])]=0
    for i in range(len(epoch1)):
        config_count[str(epoch1[i])]+=1
        
    listk=[]
    for i in range(len(list(config_count.values()))):
        listk.append(int(list(config_count.values())[i]))
    listmk=[]
    kcount={}


    # 갯수의 갯수 파악 (m_k)
    for i in range(len(listk)):
        kcount[listk[i]]=0
    for i in range(len(listk)):
        kcount[listk[i]]+=1
    for i in range(len(kcount)):
        listmk.append(kcount[sorted(list(kcount))[i]])

    return sorted(list(kcount)), listmk
    

def get_H_s(x, y):
    list100=[]
    list100kmk=[]
    for i in range(len(x)):
        list100kmk.append(x[i]*y[i])
    for i in range(len(x)):
        for j in range(list100kmk[i]):
            list100.append(x[i])
    N=len(list100)
    H_s=0
    for i in range(len(x)):
        H_s-=(x[i]*y[i]/N)*np.log(x[i]/N)
    return H_s

In [ ]:
fullconfigs=torch.tensor(pd.read_pickle('drive/MyDrive/loss_IG/Ising/lr=0.01/4*4_inf_full.pkl'))

In [ ]:
x,y=get_listmk(fullconfigs)
S=get_H_s(x, y)
S

11.090354888959125